In [ ]:
#!pip3 install extract_msg

In [41]:
import pandas as pd
import os
from tqdm import tqdm
from pathlib import Path
import multiprocessing

from docx import Document
import extract_msg
from pptx import Presentation
from openpyxl import load_workbook
import matplotlib.pyplot as plt

In [42]:
!pip install pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

def convert_pdf_to_txt(path):
    
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,
                                  caching=True, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text


In [43]:
def write_errors(path):
    with open('ERRORS.txt', 'a') as f:
        f.write(str(path)+'\n')

In [44]:
def get_text_docx(path):
    full_text = ''
    document = Document(path)
    for para in document.paragraphs:
        full_text += para.text + '. '
        #Add tables
#     for table in document.tables:
#         for i, row in enumerate(table.rows):
#             for cell in row.cells:
#                 full_text += cell.text
    return full_text

In [45]:
def get_text_pptx(path):
    full_text = ''
    prs = Presentation(path)
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                full_text += shape.text + '. '
    return full_text

In [46]:
def get_text_xlsx(path):
    full_text = ''
    wb = load_workbook(path, data_only=True)
    for sheetname in wb.sheetnames:
        ws = wb[sheetname]
        all_rows = list(ws.rows)
        for row in all_rows:
            for r in row:
                if r.value is not None:
                    full_text += str(r.value) + '. '
    return full_text

In [47]:
def get_text_msg(path):
    msg = extract_msg.openMsg(path)
    full_text = msg.subject + '. ' + msg.body
    return full_text

In [49]:

def extract_file(paths, step):
    text = []
    path = []
    for file in tqdm(paths):
        if str(file) in processeddata.paths.values:
            #print ("skipping: ")
            #print (str(file))
            continue
        #print ('Processing:')
        #print (str(file))
        
        if file.suffix == '.docx':
            try:
                text=(get_text_docx(file))
                path=(str(file))
            except:
                write_errors(file)
                #print("\nERROR:" + str(file)+ "\n")
        elif file.suffix == '.pptx':
            try:
                text=(get_text_pptx(file))
                path=(str(file))
            except:
                write_errors(file)
                #print("\nERROR:" + str(file)+ "\n")
        elif file.suffix == '.xlsx':
            try:
                text=(get_text_xlsx(file))
                path=(str(file))
            except:
                write_errors(file)
                #print("\nERROR:" + str(file)+ "\n")
        elif file.suffix == '.msg' or file.suffix == '.eml':
            try:
                text=(get_text_msg(file))
                path=(str(file))
            except:
                write_errors(file)
                #print("\nERROR:" + str(file)+ "\n")
        elif file.suffix == '.pdf':
            try:
                text=(convert_pdf_to_txt(file))
                path=(file)
            except:
                write_errors(file)
                #print("\nERROR:" + str(file)+ "\n")
                #print(str(file))
        if len(text)>0:
            print (str(file))
            from IPython.core.debugger import set_trace
            #set_trace()
            df = pd.DataFrame({'paths':[path], 'text':[text]})
            #df.to_csv('data'+str(step)+'.csv')
            #with open('/Users/rmRM/Desktop/RM/trainv4.csv', 'a') as f:
            #    df.to_csv(f, header=False)
            
            with open('/Users/rmRM/Desktop/RM/trainv4.csv','a') as f:
                f.write('\n')    
            df.to_csv('/Users/rmRM/Desktop/RM/trainv4.csv', index = False, header = False, mode='a')
            #print (df.head(10))
            #break

In [50]:
need_extention = {'.docx':0,'.doc':0,'.xls':0,'.xlsx':0,'.ppt':0,'.pptx':0,'.pdf':0,'.msg':0,'.eml':0}
#all_data = sorted(list(Path('/Users/rmRM/Desktop/Evgenii/DS').rglob('*.*')))


In [95]:
#Loading processed data
processeddata=pd.read_csv('/Users/rmRM/Desktop/RM/trainv8.csv')
processeddata.head(10)
processeddata.shape
#print (len(processeddata)-len(processeddata.drop_duplicates()))
#import textract
#text = textract.process("/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP - Retail/Meetings_Communications/Project Status Meeting Minutes/1. OFX/2015_12_03_OFX Meeting Agenda.doc")
#print (text)
#processeddata.drop_duplicates().shape


(6247, 2)

In [52]:
#Getting the list of files
from os.path import join
from glob import glob
files = []
for ext in ('*.docx', '*.doc', '*.xls','*.xlsx', '*.ppt', '*.pptx','*.pdf', '*.msg', '*.eml'):
    files.extend(list(Path('/Users/rmRM/Desktop/Evgenii/DS').rglob(ext)))

all_data = sorted(files)
len(all_data)
#print (files)


8058

In [53]:
len(all_data)



8058

In [54]:
#testfile='/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP - Retail/Meetings_Communications/Project Status Meeting Minutes/1. OFX/2015_12_03_OFX Meeting Agenda.doc'
#text=(get_text_docx(testfile))

In [55]:
#from os.path import join
#from glob import glob
#files = []
#for ext in ('*.docx', '*.doc', '*.xls','*.xlsx', '*.ppt', '*.pptx','*.pdf', '*.msg', '*.eml'):
#    files.extend(list(Path('/Users/rmRM/Desktop/Evgenii/DS').rglob(ext)))

#all_data = sorted(files)
#print (files)


#all_data = sorted(list(Path('/Users/rmRM/Desktop/Evgenii/DS').rglob('*.docx')))
#all_data = sorted(list(Path('/Users/rmRM/Desktop/Evgenii/DS').rglob('*.doc')))
#all_data = sorted(list(Path('/Users/rmRM/Desktop/Evgenii/DS').rglob('*.xls')))
#all_data = sorted(list(Path('/Users/rmRM/Desktop/Evgenii/DS').rglob('*.xlsx')))
#all_data = sorted(list(Path('/Users/rmRM/Desktop/Evgenii/DS').rglob('*.ppt')))
#all_data = sorted(list(Path('/Users/rmRM/Desktop/Evgenii/DS').rglob('*.pptx')))
#all_data = sorted(list(Path('/Users/rmRM/Desktop/Evgenii/DS').rglob('*.pdf')))
#all_data = sorted(list(Path('/Users/rmRM/Desktop/Evgenii/DS').rglob('*.msg')))
#all_data = sorted(list(Path('/Users/rmRM/Desktop/Evgenii/DS').rglob('*.eml')))
#print (files)
#all_data = sorted(list(Path('/Users/rmRM/Desktop/Evgenii/DS').rglob('*.*')))

In [56]:
 #print (list(Path('/Users/rmRM/Desktop/Evgenii/DS').rglob('*.xlsx')))

In [57]:
for path in all_data:
    if path.suffix in need_extention:
        need_extention[path.suffix] += 1

In [58]:
need_extention

{'.docx': 1269,
 '.doc': 300,
 '.xls': 249,
 '.xlsx': 1832,
 '.ppt': 121,
 '.pptx': 1804,
 '.pdf': 1043,
 '.msg': 1440,
 '.eml': 0}

In [59]:
#####a='/Users/rmRM/Desktop/Evgenii/DS/Prod Checkouts.xlsx'
#####a in processeddata.paths.values

#for path in all_data:
#    exists = path in processeddata.paths
#    print(path)
    #print (processeddata['paths'])
    #found = processeddata[processeddata.str.contains(path)]
    #if (found>0):
    #    print(found.count())

In [60]:
#len(all_data)
#remove files that were already processed:
#xlsxattributes = pd.read_excel('/Users/rmRM/Desktop/Evgenii/TextExtraction/trainv5Attributes.xlsx')
#print(xlsxattributes.shape)
#xlsxattributes.head(10)
#for row in xlsxattributes.iterrows():
#    if 'abc' in str(my_list):
#    print (row[1][3])

#print (xlsxattributes.paths)
#for path in all_data:
#    print (path)
#    if path in xlsxattributes.paths:
 #       print (path)
#        need_extention[path.suffix] += 1


In [61]:
#print (xlsxattributes.paths[0])
#print (all_data[4])
len(all_data)


8058

In [62]:
step_size = 10000
start = 0
#stop = step_size
#while stop<len(all_data):
#    extract_file(all_data[start:stop], stop)
#    start = stop
#    if stop + step_size<len(all_data):
#        stop += step_size
#    else:
#        break
extract_file(all_data[start:len(all_data)], len(all_data))
print('EVERY FILE DONE')

  1%|          | 59/8058 [00:00<00:19, 402.63it/s]

/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP - Retail/Meetings_Communications/Project Status Meeting Minutes/1. OFX/2015_12_03_OFX Meeting Agenda.docx
/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP - Retail/Meetings_Communications/Project Status Meeting Minutes/1. OFX/2015_12_11_NGP_P4_OFX Scope Review.doc
/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP - Retail/Meetings_Communications/Project Status Meeting Minutes/1. OFX/2015_12_11_NGP_P4_OFX Scope Review.docx
/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP - Retail/Meetings_Communications/Project Status Meeting Minutes/1. OFX/2015_12_18_NGP_P4_OFX .doc
/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP - Retail/Meetings_Communications/Project Status Meeting Minutes/1. OFX/2015_12_18_NGP_P4_OFX .docx
/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP - Retail/Meetings_Communications/Project Status Meeting Minutes/1. OFX/2015_12_18_NGP_P4_OFX Scope Review.doc
/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP - Retail/Meetings_Communications/Project Sta

  1%|          | 94/8058 [00:00<00:35, 226.89it/s]

/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP - Retail/Meetings_Communications/Project Status Meeting Minutes/3. Alerts Engine/2015_12_17_NGP_P4_Alerts.docx
/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP - Retail/Meetings_Communications/Project Status Meeting Minutes/4. Authentication/2015_12_02_Authentication Meeting Agenda.doc
/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP - Retail/Meetings_Communications/Project Status Meeting Minutes/4. Authentication/2015_12_02_Authentication Meeting Agenda.docx
/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP - Retail/Meetings_Communications/Project Status Meeting Minutes/4. Authentication/2015_12_11_NGP_P4_Authentication.doc
/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP - Retail/Meetings_Communications/Project Status Meeting Minutes/4. Authentication/2015_12_11_NGP_P4_Authentication.docx
/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP - Retail/Meetings_Communications/Project Status Meeting Minutes/4. Authentication/2015_12_18_NGP_P4_Authentication Minu

  2%|▏         | 151/8058 [00:00<00:31, 247.18it/s]

...

/Users/rmRM/Desktop/Evgenii/DS/Tier 5 - 6/TB-19-05 Home Equity Disclosure Booklet/Project Gating_Financials/LOE_Reclass Information/Aug19 US GL RC_Home Equity Reclass_Sales_Pod.xlsx
/Users/rmRM/Desktop/Evgenii/DS/Tier 5 - 6/TB-20-20 View Rewards USDP-Mobile/Status/Rewards Update 1-16-20.doc
/Users/rmRM/Desktop/Evgenii/DS/Tier 5 - 6/TB-20-20 View Rewards USDP-Mobile/Status/Rewards Update 1-16-20.docx
/Users/rmRM/Desktop/Evgenii/DS/Tier 5 - 6/TB-20-20 View Rewards USDP-Mobile/Status/Rewards Update 1-22-20.doc
/Users/rmRM/Desktop/Evgenii/DS/Tier 5 - 6/TB-20-20 View Rewards USDP-Mobile/Status/Rewards Update 1-22-20.docx
/Users/rmRM/Desktop/Evgenii/DS/Tier 5 - 6/TB-20-20 View Rewards USDP-Mobile/Status/Rewards Update 1-29-20.doc
/Users/rmRM/Desktop/Evgenii/DS/Tier 5 - 6/TB-20-20 View Rewards USDP-Mobile/Status/Rewards Update 1-29-20.docx
/Users/rmRM/Desktop/Evgenii/DS/Tier 5 - 6/TB-20-20 View Rewards USDP-Mobile/Status/Rewards Update 1-8-20.doc
/Users/rmRM/Desktop/Evgenii/DS/Tier 5 - 6/TB-2

100%|█████████▉| 8032/8058 [1:41:01<00:00, 69.29it/s]

/Users/rmRM/Desktop/Evgenii/DS/Tier 5 - 6/UP-17-05 - Direct Send-Electronic Payments/Meetings_Communications/Project Status Meeting Minutes/Direct Send Money Weekly DgC Status Meeting Agenda 5-25-17.doc
/Users/rmRM/Desktop/Evgenii/DS/Tier 5 - 6/UP-17-05 - Direct Send-Electronic Payments/Meetings_Communications/Project Status Meeting Minutes/Direct Send Money Weekly DgC Status Meeting Agenda 5-25-17.docx
/Users/rmRM/Desktop/Evgenii/DS/Tier 5 - 6/UP-17-05 - Direct Send-Electronic Payments/Meetings_Communications/Project Status Meeting Minutes/Direct Send Money Weekly DgC Status Meeting Agenda 6-15-17.doc
/Users/rmRM/Desktop/Evgenii/DS/Tier 5 - 6/UP-17-05 - Direct Send-Electronic Payments/Meetings_Communications/Project Status Meeting Minutes/Direct Send Money Weekly DgC Status Meeting Agenda 6-15-17.docx
/Users/rmRM/Desktop/Evgenii/DS/Tier 5 - 6/UP-17-05 - Direct Send-Electronic Payments/Meetings_Communications/Project Status Meeting Minutes/Direct Send Money Weekly DgC Status Meeting Ag

100%|██████████| 8058/8058 [1:41:01<00:00,  1.33it/s]

/Users/rmRM/Desktop/Evgenii/DS/Tier 5 - 6/UP-17-05 - Direct Send-Electronic Payments/Scope_Requirements Management/Requirements Package/PRJ96119 Draft 5 layout.xlsx
EVERY FILE DONE


In [91]:
#*******************************
#removing dups and saving
processeddata=pd.read_csv('/Users/rmRM/Desktop/RM/trainv8.csv')
processeddata.head(10)
#processeddata.shape
#newdf=processeddata.drop_duplicates()
#newdf.to_excel('trainv8.xlsx', index=False)
#newdf.to_csv('trainv8.csv', index=False)



,paths,text
0,/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/Cros...,4000 1234 5678 1234\n\n00/00\n\nTD CUSTOMER\n\n
1,/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP ...,NGP Project 4 - Core Capabilities\nProject Ma...
2,/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP ...,NGP Project 4 - Core Capabilities\nProject Ma...
3,/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP ...,"NGPR - Tablet, Browser as well as Core Capabi..."
4,/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP ...,NGP - P4 Team Roster (Core Team & Stakheholder...
5,/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/TB-1...,NGP Readiness & Conversion Status. As of July ...
6,/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/TB-1...,NGP Readiness & Conversion Status. As of July ...
7,/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/TB-1...,NGP Readiness & \nConversion Status \nAs of Ju...
8,/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/TB-1...,NGP Readiness & Conversion Status. As of July ...
9,/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/TB-1...,NGP Readiness & \nConversion Status \nAs of Ju...


In [47]:
def replaceAppribute (a):
    #*******removing
    #if a in ['Archive']: return ''
    x=''
    if 'Requirements' in a: x="Project Requirements"
    if 'PCR' in a: x="Project PCR"
    if ('Design' in a) or ("Wireframes" in a): x="Design Document" 
    if ('Steering' in a) or ('SteerCo' in a): x="Steering Committee Meeting"
    if 'Communication' in a: x="Project Communication"
    if 'SRS' in a: x="System Requirements Document"
    if 'Audit' in a: x="Project Audit"     
    if 'Planning' in a: x="Project Planning"
    if 'Governance' in a: x="Project Governance"
    if 'Discovery' in a: x="Project Discovery"
    if ('Financials' in a) or ('Invoices' in a) or ('Finances' in a) or ('Financial' in a): x="Project Financials and Invoices"
    if 'Delivery' in a: x="Project Delivery"
    if 'PCR' in a: x="Project PCR"
    if 'Release' in a: x="Release Artifacts"
    if ('Readiness' in a) or ('Pilot' in a): x="Project Readiness"
    if ('Testing' in a) or ('QA' in a): x="Quality Assurance"
    if 'Stakeholder' in a: x="Stakeholder Meeting"
    if 'NGP' in a: x="NGP"
    if 'MRDC' in a: x="Mobile deposit"
    if 'Account Opening' in a: x="Account Opening"
    if 'Fiserv' in a: x="Fiserv"
    if 'Agile Transformation' in a: x="Agile Transformation"
    if 'Billpay' in a: x="Billpay"
    if 'Zelle' in a: x="Zelle"
    if 'Mobile' in a: x="Mobile app"
    if 'Public Site' in a: x="Public Site"
    if 'Small Enhancements' in a: x="Small Enhancements"
    if ('SBB' in a) or ('Small Business' in a): x="Small Business Banking"
    if 'Meeting Minutes' in a: x="Meeting Minutes"
    if ('Approvals' in a) or ('Approve' in a): x="Project Approvals"
    if 'Closure' in a: x="Project Closure" 
    if 'Control' in a: x="Change Control"
    if 'Status' in a: x="Status Report"
    if 'eTreasury' in x: x="eTreasury"
    if 'Vendor' in a: x="Vendor Management"
    return x

In [48]:
#*******************************
#adding attributes
processeddata=pd.read_csv('/Users/rmRM/Desktop/RM/trainv8.csv')
df = pd.DataFrame(columns=['attributes', 'text', 'paths'])
dfExcel = pd.DataFrame(columns=['attributes', 'text', 'paths'])
y = []

for row in processeddata.iterrows():
  str1=''
  y=row[1]['paths'].split('/')
  for x in range(7,len(y)):
   attribute=replaceAppribute(y[x])
   if (attribute==''): continue
   if (attribute in y): continue
   y[x]=attribute
   #break
   if len (str1)==0: 
    str1+=(y[x])
   else: 
    str1+=','+y[x]
  #print (str1)
  #print (y[len(y)-1])
  #break
  #if str1=='': continue
  if len(row[1]['text'])<100 or (str1==''): continue
  values_to_add = {'attributes': str1, 'text': y[len(y)-1]+row[1]['text'][:100000], 'paths': row[1]['paths']}
  row_to_add = pd.Series(values_to_add)
  df = df.append(row_to_add,ignore_index=True)  

  values_to_add = {'attributes': str1, 'text': y[len(y)-1]+row[1]['text'][:50], 'paths': row[1]['paths']}
  row_to_add = pd.Series(values_to_add)
  dfExcel=dfExcel.append(row_to_add,ignore_index=True) 

df.to_csv('trainv8WithAttributes.csv', index=False)
dfExcel.to_excel('trainv8WithAttributes.xls', index=False)
print(df.shape)
df.head(100)

(5516, 3)


,attributes,text,paths
0,"NGP,Project Communication",2016_05_09_NGP_P4_Workbook.xlsxNGP Project 4 ...,/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP ...
1,"NGP,Project Communication",2016_05_17_NGP_P4_Workbook (Autosaved).xlsxNGP...,/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP ...
2,"NGP,Project Communication","2016_05_31_NGP_P4_Workbook.xlsxNGPR - Tablet,...",/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP ...
3,"NGP,Project Communication",old2016_03_8_NGP_P4_DRAFT_LOB Perspective HL ...,/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/NGP ...
4,"NGP,Project Delivery,Project Readiness,Status ...",NGP_Readiness and Conversion Status_07_14_2017...,/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/TB-1...
...,...,...,...
95,"NGP,Change Control,Status Report",NGP_PCR_Status_04_26_2017.pptx1. Note: Project...,/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/TB-1...
96,"NGP,Change Control,Status Report",NGP_PCR_Status_05_16_2017 vF.pptx1. Note: Proj...,/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/TB-1...
97,"NGP,Change Control,Status Report",NGP_PCR_Status_05_24_2017 vF.pptx1. Note: Proj...,/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/TB-1...
98,"NGP,Change Control,Status Report",NGP_PCR_Status_06_07_2017 vF.pptx1. Note: Proj...,/Users/rmRM/Desktop/Evgenii/DS/Tier 1 - 4/TB-1...


In [49]:
import numpy as np
#*****************************
#Clean up attributes
#Generate numerical labels
tmp_list = []
for line in dfExcel['attributes'].values:
  if ',' in line:
    for l in line.split(','):
      if len(l)>2:
        tmp_list.append(l.replace(' ', ''))
    # tmp_list += [l.replace(' ', '') for l in line.split(',') if len(line)>2]
  else:
    if len(line)>2:
      tmp_list.append(line.replace(' ', ''))
print(len(set(tmp_list)))
#print(set(tmp_list))

label_map = {name:k for k,name in enumerate(set(tmp_list))}
#print (label_map)

#************************************
label_map_inv = {k:name for k,name in enumerate(label_map)}
y = []
for line in dfExcel['attributes'].values:
  target = [0 for i in range(len(label_map))]
  for l in line.split(','):
    l = l.replace(' ','')
    if l in label_map:
      target[label_map[l]]=1
  y.append(target)
y = np.stack(y)
print(y.shape)
print (y[:10])
#************************************
for key in label_map_inv:
  print(label_map_inv[key], sum(y[:,key]))


32
(5516, 32)
[[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1]]
PublicSite 202
ProjectClosure 167
ProjectPCR 328
SmallEnhancements 395
VendorManagement 60
ProjectDelivery 318
ChangeControl 1001
StakeholderMeeting 71
MeetingMinutes 166
ProjectRequirements 729
Zelle 163
ProjectCommunication 1051
ProjectAudit 17
SmallBusinessBanking 311
ProjectPlanning 27
SteeringCommitteeMeeting

In [50]:
#Calculate mean len for text data
docs = df['text'].values
mean_len = []
for doc in docs:
    mean_len.append(len(doc))
    #print (len(doc))
print('mean_len = ', sum(mean_len)/len(mean_len))

mean_len =  13904.881073241479


# Concat all

In [ ]:
data_csv = sorted(list(Path('./Data_done/').rglob('data*.csv')))[1:]
len(data_csv)

In [ ]:
train_data = sorted(list(Path('./Train_data/').rglob('data*.csv')))
len(train_data)

In [ ]:
combined_csv = pd.concat([pd.read_csv(f) for f in data_csv+train_data ])

In [ ]:
combined_csv.shape

In [ ]:
combined_csv.head()

In [ ]:
combined_csv.columns
del combined_csv['Unnamed: 0']

In [ ]:
combined_csv.drop_duplicates().shape

In [ ]:
done_paths = combined_csv['paths'].values

In [ ]:
combined_csv.to_excel('train.xlsx', index=False)

# Statistic

In [ ]:
combined_csv['len_text'] = combined_csv['text'].apply(lambda x: len(x))

In [ ]:
combined_csv[combined_csv['len_text']>50000].shape

In [ ]:
done_paths = combined_csv['paths'].values

In [ ]:
done_paths[0]

In [ ]:
combined_csv['paths'].shape

In [ ]:
len(set(combined_csv['paths'].values.tolist()))

In [ ]:
need_extention_done = {'.docx':0,'.doc':0,'.xls':0,'.xlsx':0,'.ppt':0,'.pptx':0,'.pdf':0,'.msg':0,'.eml':0}

In [ ]:
for path in done_paths:
    if '.'+path.strip().split('.')[-1] in need_extention_done:
        need_extention_done['.'+path.strip().split('.')[-1]] += 1

In [ ]:
need_extention_done